In [1]:
import numpy as np
from gurobipy import *

prng = np.random.RandomState(123)

In [2]:
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        #yield l[i:i + n]
        yield slice(i,i+n)

def torange(item):
    return list(range(item.start, item.stop, item.step if item.step is not None else 1))

The data and the Feature queries

In [3]:
x = [10, 15, 20, 23, 41, 72, 55, 50, 88, 72, 40, 18, 10, 9, 23, 32]
n = len(x)

# number of subfeatures
p = 3

# Feature queries
F1 = list(chunks(range(n), n))
F2 = list(chunks(range(n), 4))
F3 = list(chunks(range(n), 1))

x1_q = np.sum([x[f] for f in F1],axis=1)
x2_q = np.sum([x[f] for f in F2],axis=1)
x3_q = np.sum([x[f] for f in F3],axis=1)

In [4]:
print('x1: %s' % ', '.join(map(str, x1_q)))
print('x2: %s' % ', '.join(map(str, x2_q)))
print('x3: %s' % ', '.join(map(str, x3_q)))

x1: 578
x2: 68, 218, 218, 74
x3: 10, 15, 20, 23, 41, 72, 55, 50, 88, 72, 40, 18, 10, 9, 23, 32


In [5]:
eps = 1.0
deltaQ = 1

x1_lap = x1_q + prng.laplace(scale=deltaQ/eps, size=len(x1_q))
x2_lap = x2_q + prng.laplace(scale=deltaQ/eps, size=len(x2_q))
x3_lap = x3_q + prng.laplace(scale=deltaQ/eps, size=len(x3_q))

In [6]:
print('Lap x1: %s' % ', '.join(map(str, np.round(x1_lap,2))))
print('Lap x2: %s' % ', '.join(map(str, np.round(x2_lap,2))))
print('Lap x3: %s' % ', '.join(map(str, np.round(x3_lap,2))))

Lap x1: 578.5
Lap x2: 67.44, 217.21, 218.11, 74.58
Lap x3: 9.83, 18.26, 20.46, 22.96, 40.76, 71.62, 55.61, 49.87, 85.87, 71.77, 40.65, 16.99, 8.95, 9.07, 23.07, 32.31


In [7]:
opt_type = GRB.CONTINUOUS
# The size of the paritions
m1, m2, m3 = len(x1_lap), len(x2_lap), len(x3_lap)
model = Model()

# Variables
x1_dot = model.addVars(m1, lb=[0]*m1, vtype=opt_type)  # 1
x2_dot = model.addVars(m2, lb=[0]*m2, vtype=opt_type)  # 4
x3_dot = model.addVars(m3, lb=[0]*m3, vtype=opt_type)  # 16

# Constraints
model.addConstr(quicksum(x3_dot) == x1_dot[0]) 
model.addConstr(quicksum(x2_dot) == x1_dot[0])

# Equality constraints between x3 groups and x2
for i in range(m2):
    model.addConstr(quicksum(x3_dot.values()[F2[i]]) == x2_dot[i])

# Objective
obj = 1/m1 * (x1_dot[0] - x1_lap[0]) * (x1_dot[0] - x1_lap[0])
for i in range(m2):
    obj += 1/m2 * (x2_dot[i] - x2_lap[i]) * (x2_dot[i] - x2_lap[i])
for i in range(m3):
    obj += 1/m3 * (x3_dot[i] - x3_lap[i]) * (x3_dot[i] - x3_lap[i])

# Solve
model.setObjective(obj)
model.setParam('OutputFlag', False)
model.setParam('OptimalityTol', 1e-6)
model.optimize()

Academic license - for non-commercial use only


In [8]:
x1_hat = np.round([x1_dot[i].x for i in range(m1)]).astype(int)
x2_hat = np.round([x2_dot[i].x for i in range(m2)]).astype(int)
x3_hat = np.round([x3_dot[i].x for i in range(m3)]).astype(int)
               
print('Hat x1: %s' % ', '.join(map(str, x1_hat)))
print('Hat x2: %s' % ', '.join(map(str, x2_hat)))
print('Hat x3: %s' % ', '.join(map(str, x3_hat)))

Hat x1: 578
Hat x2: 68, 218, 218, 75
Hat x3: 9, 17, 20, 22, 41, 72, 56, 50, 87, 73, 41, 18, 9, 9, 23, 33


In [9]:
print('L2 Err Lap %.2f' % np.linalg.norm(x3_lap - x, 2))
print('L2 Err HAt %.2f' % np.linalg.norm(x3_hat - x, 2))

L2 Err Lap 4.32
L2 Err HAt 3.46
